In [1]:
#Enter bucket name
bucket = 'sagemaker-ml-pipeline'
prefix = 'sagemaker/heart'

#Enter data file name (e.g. heart.csv)
data_key = 'data.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [1]:
import pandas as pd
import json

# read the data from S3
heart_data = pd.read_csv(data_location)

#print out a sample of data.
# heart_data.head()

In [3]:
import numpy as np
vectors = np.array(heart_data).astype('float32')

#target column - value must be either 0 or 1
labels = vectors[:,13]
print ("label data is")
print (labels)


#drop the target column.  Use the features as part of the training data
training_data = vectors[:, :10]



label data is
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Training data is
[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. .

Let's go ahead and upload the training data to S3

In [4]:
import io
import os
import sagemaker.amazon.common as smac

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, training_data, labels)
buf.seek(0)

key = 'prediction-node-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))




/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


uploaded training data location: s3://sagemaker-ml-pipeline/sagemaker/heart/train/recordio-pb-data


## Training Artifacts
Once data is trained, it will be uploaded to the following location.

In [5]:
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://sagemaker-ml-pipeline/sagemaker/heart/output


We will do a binary classification (patient either has heart disease or not), train the model on the specified compute (e.g. c4.xlarge), and we will sepcify the features or dimiensions in our training set.

In [6]:
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker

container = get_image_uri(boto3.Session().region_name, 'xgboost-learner', "latest")

sess = sagemaker.Session()
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=13,
                           predictor_type='binary_classifier',
                           mini_batch_size=100)

linear.fit({'train': s3_train_data})

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-04-26 23:48:54 Starting - Starting the training job...
2022-04-26 23:49:21 Starting - Preparing the instances for trainingProfilerReport-1651016934: InProgress
.........
2022-04-26 23:50:47 Downloading - Downloading input data...
2022-04-26 23:51:23 Training - Downloading the training image...
2022-04-26 23:51:54 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[04/26/2022 23:51:46 INFO 140612709816128 integration.py:636] worker started
[04/26/2022 23:51:46 INFO 140612709816128] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', '

In [7]:
heartdisease_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

----------!

In [10]:
from sagemaker.predictor import csv_serializer, json_deserializer

# heartdisease_predictor.content_type = 'text/csv'
heartdisease_predictor.serializer = csv_serializer
heartdisease_predictor.deserializer = json_deserializer

In [11]:
print('Endpoint name: {}'.format(heartdisease_predictor.endpoint))

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Endpoint name: linear-learner-2022-04-26-23-52-37-369


Let's try passing the following sample data for testing. This is a single record from the file

In [12]:
vectors[5][0:13]

array([ 57. ,   1. ,   0. , 140. , 192. ,   0. ,   1. , 148. ,   0. ,
         0.4,   1. ,   0. ,   1. ], dtype=float32)

In [13]:
result = heartdisease_predictor.predict(vectors[5][0:13])
print(result)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'predictions': [{'score': 0.8270459771156311, 'predicted_label': 1}]}


In [14]:
# import sagemaker

# sagemaker.Session().delete_endpoint(heartdisease_predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
